# Single Agent Dynamics

## Rust (1987) 
*Optimal Replacement of GMC Bus Engines: An Empirical Model of Harold Zurcher*

# Slide title

- **State**: mileage of the bus

    $$x_t \in \{1, ..., 10\}$$
    
- **State transitions**: with probability $\lambda$ the mileage of the bus increases
    
    $$
    x_{t+1} = \begin{cases}
    \min \left\{ x_t + 1,10 \right\}  & \text { with probability } \lambda \\ 
    x_t & \text { with probability } 1 - \lambda
    \end{cases}
    $$
    
    Note that $\lambda$ does not depend on the value of the state


In [63]:
# Import packages
using Distributions

# Set parameters
theta = [0.13; -0.004; 3.1]
lambda = 0.82
beta = 0.95;

In [64]:
# State space: x (dim = 11) and draws from delta_e = e1 - e0 (dim = 10000)
x = 0:10
draws_e = 10000
delta_e = sort(rand(Gumbel(0,1), draws_e) - rand(Gumbel(0,1), draws_e)); 

# Static Utility
U = [- theta[1]*x - theta[2]*x.^2 - theta[3]*ones(11,1)]

# Initial value functions (state space: 11(x) x draws_e(Delta_e))
V = ones(11, draws_e)
I = ones(11, draws_e);

In [72]:
Test

LoadError: ArgumentError: Package interact not found in current path:
- Run `import Pkg; Pkg.add("interact")` to install the interact package.


In [ ]:




rng(1)



% Iterate the Bellman equation
dist = 100;
n = 1;

tic
while dist>1e-20
    
    % Expected future values (mean over possible shocks)
    Exp_V = repmat(mean(V,2) ,1,draws_e);
    
    % Calculate new value function
    V_0_new = U(:,1)*ones(1,draws_e) + ones(11,1)*delta_e + ...
        beta*( (1-lambda)*Exp_V + lambda*[Exp_V(2:11,:); Exp_V(11,:)] );
    V_1_new = U(:,2)*ones(1,draws_e) + ...
        beta*( (1-lambda)*repmat(Exp_V(1,:) ,11,1) + lambda*repmat(Exp_V(2,:) ,11,1) );
    V_new = max(V_0_new, V_1_new);

    % Update distance for convergence
    dist = max(max(abs(V_new - V)));
    n = n+1;
    
    % Update value function
    V = V_new;
    
end
toc

% Calculate the policy function (I==1: invest/replace the engine)
I = V_1_new>V_0_new;

% Check frequency of investment decisions
sum(sum(I))/(11*draws_e)